<a href="https://colab.research.google.com/github/syedshubha/TeachingQuantumComputing/blob/main/QKD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --quiet qiskit qiskit-aer pylatexenc &> /dev/null

In [2]:
from qiskit import *
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector, Pauli

**Basis Compatibility Overview**

In [3]:
from numpy import sqrt, pi
Basis = [Pauli('Z'), Pauli('X'),'Z','X']

# [|0⟩,|1⟩,|+⟩,|-⟩]
States = [Statevector([1, 0]), Statevector([0, 1]),
          Statevector([1/sqrt(2), 1/sqrt(2)]), Statevector([1/sqrt(2),-1/sqrt(2)])]
state = ['|0⟩','|1⟩','|+⟩','|-⟩']

In [4]:
for i in range(4):
  for j in range(2):
    val = States[i].expectation_value(Basis[j])
    print(f"⟨{Basis[j+2]}⟩ w.r.t. {state[i]} is {val:.1f}")

⟨Z⟩ w.r.t. |0⟩ is 1.0
⟨X⟩ w.r.t. |0⟩ is 0.0
⟨Z⟩ w.r.t. |1⟩ is -1.0
⟨X⟩ w.r.t. |1⟩ is 0.0
⟨Z⟩ w.r.t. |+⟩ is 0.0
⟨X⟩ w.r.t. |+⟩ is 1.0
⟨Z⟩ w.r.t. |-⟩ is 0.0
⟨X⟩ w.r.t. |-⟩ is -1.0


You can see both $Z$ and $X$ has eigenvalues $\{-1,+1\}$, which we find when measure in exactly their eiegenbasis. When we measure in incompatible basis, we get 0, meaning we measure +1 and -1 with 50-50 probability.

# B92 Protocol

In [5]:
n = 100 # You can change this
from random import choices

Alice sends |0⟩ if her basis bit = 0

She sends |+⟩ when the basis bit = 1



In [6]:
alice_basis = choices([0, 1], k=n) # Alice generates her random basis bits

In [7]:
def b92_encoding(bits):
  states = []
  for i in range(n):
    q = QuantumCircuit(1,1)
    if bits[i] ==1:
      q.h(0)
    states.append(q)
  return states

Bob measures Z if his basis bit = 0

Bob measures X if his basis bit = 1


If Bob measures |1⟩, he knows Alice sent |+⟩

If Bob measures |-⟩, he knows Alice sent |0⟩

Bob also notes the positions where he knows what Alice sent. Note, if Bob gets |0⟩ or |+⟩, Bob cannot tell! (**Why?**)

In [8]:
bob_basis = choices([0, 1], k=n) # Bob generates his random basis bits

In [9]:
def b92_decoding(basis, states):
  bob_key = ""
  public = []
  for i in range(n):
    if basis[i] == 1:
      states[i].h(0)
    states[i].measure(0,0)
    counts = Aer.get_backend('aer_simulator').run(states[i], shots=1).result().get_counts()
    if  list(counts.keys())[0] == '1':
      public.append(i)
      bob_key = bob_key + str(1-basis[i]) # Can you tell why we appended 1-basis[i]?
  return bob_key, public

If there is Eve between them, she can also guess a basis and measure the qubits

In [10]:
eve_basis = choices([0, 1], k=n) # Eve generates her random basis bits

In [11]:
def eavesdropping(basis, states):
  for i in range(n):
    if basis[i] == 1:
      states[i].h(0)
    states[i].measure(0,0)
  return states

Now Bob anounce the position so that Alice can make her key.

In [12]:
def classical_comm(public,alice_bits):
  alice_key = ""
  for i in public:
    alice_key = alice_key + str(alice_bits[i])
  return alice_key

**Now here is the full protocol without Eavesdropper**

In [13]:
Q = b92_encoding(alice_basis)
bob_key, public = b92_decoding(bob_basis, Q)
alice_key = classical_comm(public,alice_basis)

print(alice_key)
print(bob_key)
print(alice_key == bob_key)

0010001110010000010111
0010001110010000010111
True


**This is B92 with someone eavesdropping**

In [14]:
#Q = b92_encoding(alice_basis)
Q = eavesdropping(eve_basis, Q)
bob_key, public = b92_decoding(bob_basis, Q)
alice_key = classical_comm(public,alice_basis)

print(alice_key)
print(bob_key)
print(alice_key == bob_key)

011100011111101010011101101000101110110
011100000000111000100110010000011011001
False


You see, they don't match! In practice, we can even just match a small portion to detect eavesdropping

# BB84 Protocol

Alice now needs two random bits to encode states.

$|\psi_{00}\rangle = |0\rangle, |\psi_{01}\rangle = |1\rangle, |\psi_{10}\rangle = |+\rangle, |\psi_{11}\rangle = |-\rangle$

In [15]:
def bb84_encode(basis, eig):
  QC = []
  for i in range(n):
    qc = QuantumCircuit(1,1)
    if eig[i]==1:
      qc.x(0)
    if basis[i]==1:
      qc.h(0)
    QC.append(qc)
  return QC

In [16]:
alice_eig = choices([0, 1], k=n) # Alice chooses eigenvectors according to this

Bob now can decode quite easily! Just guess basis and measure.

In [17]:
def bb84_decode(basis, QC):
  for i in range(n):
    if basis[i]==1:
      QC[i].h(0)
    QC[i].measure(0,0)
    bob_result = ""
  for i in range(n):
      counts = Aer.get_backend('aer_simulator').run(QC[i], shots=1).result().get_counts()
      bob_result = bob_result + list(counts.keys())[0]
  return bob_result

Now we need to match both basis to generate the key at both end. In B92, Bob generates the key first, but here both generates simultaneously. Eavesdropping remains same as before.

In [18]:
def key(alice_basis, bob_basis, alice_eig, bob_result):
  match = []
  alice_key = ""
  bob_key = ""
  for i in range(n):
    if alice_basis[i]==bob_basis[i]:
      match.append(i)
      alice_key = alice_key + str(alice_eig[i])
  for i in match:
    bob_key = bob_key + bob_result[i]
  return alice_key, bob_key

**Now here is the full protocol without Eavesdropper**

In [19]:
Q = bb84_encode(alice_basis, alice_eig)
bob_result = bb84_decode(bob_basis, Q)
alice_key, bob_key = key(alice_basis, bob_basis, alice_eig, bob_result)
print(alice_key)
print(bob_key)
print(alice_key == bob_key)

0011010110101000010110011100101001111000111100001001010
0011010110101000010110011100101001111000111100001001010
True


**This is BB84 with someone eavesdropping**

In [20]:
#Q = bb84_encode(alice_basis, alice_eig)
Q = eavesdropping(eve_basis, Q)
bob_result = bb84_decode(bob_basis, Q)
alice_key, bob_key = key(alice_basis, bob_basis, alice_eig, bob_result)
print(alice_key == bob_key)

False


Similarly, in practice, we will just match a small portion to detect eavesdropping

# Demonstration of a Bell Test

Say $A_1,A_2,B_1,B_2$ can have values +1 or -1.

Define $S= A_1.(B_1+B_2)+A_2.(B_1-B_2)$

In [21]:
from itertools import product
S = []
for a1,a2,b1,b2 in product([-1,1], repeat=4):
  s = a1*(b1+b2) + a2*(b1-b2)
  if s not in S:
    S.append(s)
print(S)

[2, -2]


So S is either +2 or -2.

Now if we calculate the expected value, CHSH = |⟨S⟩|, it can at best take the value 2 (when every S is only +2 or only -2), and 0 if there is equal numbers of +2 and -2. Hence $$ |⟨S⟩| \leq 2 $$
This is called CHSH inequality.

Take $A_1=Z, A_2=X,B_1=\frac{1}{\sqrt{2}}(Z+X), B_2=\frac{1}{\sqrt{2}}(Z-X)$

Can you see they all have (eigen)values +1 or -1 when you measure them? [Hint: their Trace = 0, Determinant = 1].

Let's now define S similarly, $$S= A_1\otimes(B_1+B_2)+A_2\otimes(B_1-B_2)$$


Then you can easily show, $CHSH = |\langle S \rangle| = |\sqrt{2} (\langle Z\otimes Z \rangle + \langle X\otimes X \rangle)| $


In [22]:
def CHSH(state):
  XX = state.expectation_value(Pauli('XX'))
  ZZ = state.expectation_value(Pauli('ZZ'))
  CHSH = abs((sqrt(2))*(XX+ZZ))
  print(CHSH)

let's test this for a product state, you can change the qubit states.

In [23]:
qubit_1 = Statevector([0.6, 0.8]) # you can change it
qubit_2 = Statevector([1, 0]) # you can change it

state = qubit_1.tensor(qubit_2)
CHSH(state) # it will always print <2

0.3959797974644668


Any LHVT will respect this inequality.

Now test this for Bell state: $\frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$

In [24]:
bell_state = Statevector([1/sqrt(2), 0, 0, 1/sqrt(2)])
CHSH(bell_state)

2.82842712474619


You see $CHSH > 2$, so Bell state violates the CHSH inequality. This is why we cannot explain quantum mechanics with any local hidden variable theory. Physics Nobel 2022 was awarded for this!

Now we have the famous E91 QKD protocol based on this! I initially wanted to keep this as an exercise because I am lazy, but then I let gemini complete it for me. You can have a look!

First, we generate pairs of entangled qubits (Bell States: $|\Phi^+\rangle$)

In [25]:
alice_basis = choices([0, 1, 2], k=n)
bob_basis = choices([0, 1, 2], k=n)

def e91_source(n):
    states = []
    for i in range(n):
        qc = QuantumCircuit(2, 2)
        qc.h(0)
        qc.cx(0, 1)
        states.append(qc)
    return states

Unlike BB84, E91 relies on specific measurement angles to verify the CHSH inequality.

Alice uses: $0^\circ$ (Z), $45^\circ$ (W), $90^\circ$ (X).

Bob uses: $45^\circ$ (W), $90^\circ$ (X), $135^\circ$ (V).

In [26]:
def e91_measure(states, alice_b, bob_b):
    results = []

    for i in range(n):
        qc = states[i]

        # --- Apply Alice's Basis Rotation (Qubit 0) ---
        if alice_b[i] == 0:   # Z-basis (0 deg)
            pass              # No gate needed
        elif alice_b[i] == 1: # X-basis (90 deg)
            qc.h(0)
        elif alice_b[i] == 2: # W-basis (45 deg)
            qc.ry(-pi/4, 0)   # Rotate by -pi/4

        # --- Apply Bob's Basis Rotation (Qubit 1) ---
        if bob_b[i] == 0:     # W-basis (45 deg)
            qc.ry(-pi/4, 1)
        elif bob_b[i] == 1:   # X-basis (90 deg)
            qc.h(1)
        elif bob_b[i] == 2:   # V-basis (135 deg)
            qc.ry(-3*pi/4, 1)

        # Measure both
        qc.measure(0, 0) # Alice
        qc.measure(1, 1) # Bob

        # Run simulation
        counts = Aer.get_backend('aer_simulator').run(qc, shots=1).result().get_counts()
        result_bit_string = list(counts.keys())[0] # e.g., '01'
        results.append(result_bit_string)

    return results

We generate a key only when Alice and Bob use the same measurement axis (parallel bases).

Alice 1 (X) matches Bob 1 (X).

Alice 2 (W) matches Bob 0 (W).

In [27]:
def generate_key(alice_b, bob_b, results):
    alice_key = ""
    bob_key = ""
    matches = 0

    for i in range(n):
        # Qiskit results are 'BobAlice' (Little Endian)
        # result '01' means Alice=1, Bob=0
        res = results[i]
        val_alice = res[1]
        val_bob = res[0]

        # Check for matching bases
        # Alice 1 (X) == Bob 1 (X)
        # Alice 2 (W) == Bob 0 (W)
        if (alice_b[i] == 1 and bob_b[i] == 1) or (alice_b[i] == 2 and bob_b[i] == 0):
            alice_key += val_alice
            bob_key += val_bob
            matches += 1

    return alice_key, bob_key, matches

The remaining mismatched bases are not wasted; they are used to calculate the CHSH. If $CHSH > 2$, it proves the system is quantum (entangled) and no eavesdropper intercepted the particles.

In [28]:
def check_chsh(alice_b, bob_b, results):
    # Counts for correlation E(a, b)
    # E = (N_same - N_diff) / Total
    # We need specific combinations for S = E(A0, B0) - E(A0, B2) + E(A2, B0) + E(A2, B2)

    # Dictionary to store counts: keys are (alice_base, bob_base)
    # values are [same_count, diff_count]
    data = {}

    for i in range(n):
        a_base = alice_b[i]
        b_base = bob_b[i]

        # Parse result
        res = results[i]
        val_alice = res[1]
        val_bob = res[0]

        if (a_base, b_base) not in data:
            data[(a_base, b_base)] = [0, 0]

        if val_alice == val_bob:
            data[(a_base, b_base)][0] += 1 # Same
        else:
            data[(a_base, b_base)][1] += 1 # Different

    # Helper to calculate Correlation E
    def get_E(a, b):
        if (a, b) not in data: return 0
        same, diff = data[(a, b)]
        if same + diff == 0: return 0
        return (same - diff) / (same + diff)

    # Calculate S statistics (using bases A0, A2 and B0, B2 from our defined sets)
    # In our mapping:
    # A0=Z, A2=W
    # B0=W, B2=V

    E_a0_b0 = get_E(0, 0)
    E_a0_b2 = get_E(0, 2)
    E_a2_b0 = get_E(2, 0)
    E_a2_b2 = get_E(2, 2)

    # CHSH formula
    S = E_a0_b0 - E_a0_b2 + E_a2_b0 + E_a2_b2
    return abs(S)

# --- Execution ---
qc_list = e91_source(n)
raw_results = e91_measure(qc_list, alice_basis, bob_basis)
a_key, b_key, count = generate_key(alice_basis, bob_basis, raw_results)
s_val = check_chsh(alice_basis, bob_basis, raw_results)

print(f"Key Length: {count}")
print(f"Key Match: {a_key == b_key}")
print(f"CHSH Value: {s_val:.4f} (Should be ~2.82 for pure quantum)")

Key Length: 32
Key Match: True
CHSH Value: 2.0143 (Should be ~2.82 for pure quantum)
